In [ ]:
from keras.activations import silu, swish, linear, tanh, leaky_relu, sigmoid, relu, elu, selu, gelu, linear
from keras.layers import Input, LSTM, Dense, GaussianNoise, BatchNormalization, Activation, PReLU, SimpleRNN
from keras.initializers.initializers_v2 import RandomNormal, RandomUniform
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as ex
import plotly.io as pio
from keras.regularizers import l1, l2, L1L2
from keras.optimizers import nadam_v2, rmsprop_v2, adamax_v2
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
from keras.models import Sequential
import onnx
import tf2onnx
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from sklearn.model_selection import train_test_split
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, KernelPCA,IncrementalPCA,SparsePCA
from joblib import dump, load
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from datetime import datetime
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import glob
import os
from scipy.signal import savgol_filter
import numpy as np
import seaborn as sn
%load_ext tensorboard
from tensorflow import keras 

In [ ]:
model_version = 'analysis_version'


In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)


In [ ]:
# Directories for expreiments .csvs
source_dir = 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/row_files/combines/smoothed/With_cooling_feature_for_Model_training_5s/'
testdata_dir = 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/row_files/combines/smoothed/With_cooling_feature_for_Model_training_5s/Testing_data/'
preprocessor_dir = 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\csvs\\smoothed\\AI_Model\\Backup\\Z_Welle\\' + model_version + '\\'
model_dir = 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\csvs\\smoothed\\AI_Model\\Backup\\Z_Welle\\' + model_version + '\\'
analysis_path= 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Analysis_&_feature_Engineering\\'
log_dir = "C:/Temperaturkompensation/logs/" + \
    datetime.now().strftime("%Y%m%d-%H%M%S")
weights_path = str(model_dir+"/weights")
image_path = 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\csvs\\smoothed\\AI_Model\\Backup\\Z_Welle\\' + model_version + '\\'


In [ ]:
import onnx
def load_model(path):
    model = onnx.load_model(path)
    return model


In [ ]:
from keras.losses import LossFunctionWrapper, mean_absolute_error, mean_squared_error
import tensorflow as tf
import keras.backend as k


def max_loss(y_true: tf.Tensor, y_pred: tf.Tensor):
    error = tf.add(
                    tf.reduce_max(tf.abs((y_true - y_pred))),
                    mean_absolute_error(y_true, y_pred))
    return error

def smoothed_max_loss(y_true: tf.Tensor, y_pred: tf.Tensor):
    a=1.5
    diffs= tf.abs((y_true - y_pred))
    muls= tf.multiply(a, diffs)
    exps= tf.exp(muls)
    weighted_muls= tf.multiply(diffs, exps)
    nominater= tf.reduce_sum(weighted_muls)
    denominator= tf.reduce_sum(exps)
    boltzmann_operator= nominater/denominator
    error = tf.add(boltzmann_operator, mean_absolute_error(y_true, y_pred))
    return error

def max_loss_v1(y_true: tf.Tensor, y_pred: tf.Tensor):
    error = tf.add(tf.multiply(tf.reduce_max(tf.abs((y_true - y_pred))), 1), tf.sqrt(mean_squared_error(y_true, y_pred)))
    return error

def max_loss_v2(y_true: tf.Tensor, y_pred: tf.Tensor):
    error = tf.add(tf.pow(tf.reduce_max(tf.abs((y_true - y_pred))),2), mean_squared_error(y_true, y_pred))
    return error

def max_loss_v3_mae_std(y_true: tf.Tensor, y_pred: tf.Tensor):
    mean_ae= mean_absolute_error(y_true, y_pred)
    #vars= tf.pow(tf.subtract(tf.abs(y_true - y_pred),mean_ae),2)
    vars= tf.abs(tf.subtract(tf.abs(y_true - y_pred),mean_ae))
    errors_std= tf.reduce_mean(vars)
    #error = tf.add(mean_squared_error(y_true, y_pred), errors_std)
    error = tf.add(mean_ae, errors_std)
    return error

def loss_mse_vars(y_true: tf.Tensor, y_pred: tf.Tensor):
    mean_ae= mean_absolute_error(y_true, y_pred)
    variances= tf.pow(tf.subtract(tf.abs(y_true - y_pred),mean_ae),2)
    var= tf.reduce_mean(variances)
    error = tf.add(mean_squared_error(y_true, y_pred), var)
    return error

# def calc_fidelity(y_true: tf.Tensor, y_pred: tf.Tensor):
#     # session= k.get_session()
#     # y_true_array= session.run(y_true)
#     # y_pred_array= session.run(y_pred)
#     # cc= np.corrcoef(y_true_array,y_pred_array)
#     NRMSE= tf.divide(tf.sqrt(mean_squared_error(y_true, y_pred)),tf.subtract(tf.reduce_max(y_true),tf.reduce_min(y_true)))
#     fidelity= tf.abs(tf.multiply(100.0, tf.subtract(1.0,NRMSE)))
#     return fidelity
    
special_loss = LossFunctionWrapper(smoothed_max_loss)
#fidelity_wrapper= LossFunctionWrapper(calc_fidelity)


In [ ]:
### window  represents time period by each entry in the buffer
### Shift represents the jump from value to next one in the buffer
### sampling rate 
def generateDataSource(signal_input=None, input_columns: list = [], output_length: int = 1, signal_output=None, window=1, shift=1, sample_rate=1):
    #subsequence_len= (window -1) *shift + 1
    subsequence_len= (window) *shift
    Signal_Length = signal_input.shape[0]
    #num_samples = int((Signal_Length - (window-1) * shift))
    #num_samples = int((Signal_Length - window + 1) / shift)
    num_samples = 1 + int((Signal_Length - subsequence_len) / sample_rate)
    x = np.zeros(shape=(num_samples, window, signal_input.shape[1]))
    y = np.zeros(shape=(num_samples, output_length, 1))
    for i in range(num_samples):
        x[i] = np.asarray([signal_input[i*sample_rate + j * shift] for j in range(0,window)])
        y[i] = signal_output[i*sample_rate + (window-1) * shift :i*sample_rate+ (window-1) * shift + output_length]
#        print('x[{}]= {}'.format(i, x[i]))
#        print('y[{}]= {}'.format(i, y[i]))
    return x, y

In [ ]:
x= np.asarray([
    [1,1,1,1,1],
    [2,2,2,2,2],
    [3,3,3,3,3],
    [4,4,4,4,4],
    [5,5,5,5,5],
    [6,6,6,6,6],
    [7,7,7,7,7],
    [8,8,8,8,8],
    [9,9,9,9,9],
    [10,10,10,10,10],
    [11,1,1,1,1],
    [12,2,2,2,2],
    [13,3,3,3,3],
    [14,4,4,4,4],
    [15,5,5,5,5],
    [16,6,6,6,6],
    [17,7,7,7,7],
    [18,8,8,8,8],
    [19,9,9,9,9],
    [100,10,10,10,10]
])
y= np.asanyarray([11,22,33,44,55,66,77,88,99,1010,11,22,33,44,55,66,77,88,99,1010])
generateDataSource(signal_input=x,output_length=1, signal_output=y,window=3,shift=2,sample_rate=1)

In [ ]:
# Implement special preprocessing steps
# 1 calculate new Feature
# 2 make rounding


from sklearn.preprocessing import FunctionTransformer


def special_preprocess(data: np.ndarray):
    delta_t_spindel_bett = data[:, 1] - data[:, 0]
    delta_t_spindel_bett = delta_t_spindel_bett.reshape((-1, 1))
    result = np.concatenate((data[:, 2:5], delta_t_spindel_bett), axis=1)
    return result


def rounding(signals: np.ndarray):
    ##rounding t_motor to nearest integer
    signals[:,1]= np.round(signals[:,1])
    signals= np.round(signals, 2)
    return signals

def rounding2(signals: np.ndarray):
    return np.round(signals, 5)


def remove_t_bett(data: np.ndarray):
    result = data[:, 1:]
    result[:, 0] = result[:, 0] - data[:, 0]
    result[:, 1] = result[:, 1] - data[:, 0]
    return result

def remove_t_bett_not_delete(data: np.ndarray):
    result = data.copy()
    result[:, 1] = result[:, 1] - result[:, 0]
    result[:, 2] = result[:, 2] - result[:, 0]
    return result

data = np.asarray([[24, 90, 30, 500, 0, 0, 0, 0],
                   [24, 90, 30, 500, 0, 0, 0, 0],
                   [24, 90, 30, 500, 0, 0, 0, 0],
                   [24, 90, 30, 500, 0, 0, 0, 0]])
data.shape
print(remove_t_bett(data))
print(data)

window = 60
shift = 6
sampling_rate= 6

In [ ]:
# selected_Columns= ['diff', 'drhz','t_bett',]#,'t_kluelung']# , 'diff'
# [ 't_bett','t_spindel', 't_y_achse', 't_z_achse', 'drhz', 't_motor','t_kss_tank', 't_raum', 't_spindel_vor', 't_spindel_rueck','t_schwenkantrieb']

output_variable = 'welle_z_iterpolated'#'welle_z_upsampled'#'z_welle_ok'  #'z_tcp_ok'# 'welle_z'#''#'smoothed_z_welle_ok'#
smoothed_output= 'smoothed_welle_z_iterpolated'
scaler = StandardScaler(with_mean=True, with_std=True)
minmax = MinMaxScaler(feature_range=(-1, 1))
pca = PCA(n_components=6)
#pca= IncrementalPCA(n_components=6)
# Data Parameters
selected_Columns = ['t_bett', 't_spindle','t_motor','M8','M121', 'M127', 'M7']#['t_bett', 't_spindle','t_motor', 'M8','M121', 'M127', 'M7']
#,('smoothing_data', FunctionTransformer(smoothing_data))
proposed_pipline = Pipeline(steps=[('t_bett_removal', FunctionTransformer(remove_t_bett)), ('rounding', FunctionTransformer(rounding)), ('stdscaler', scaler),('pca', pca), ('rounding2', FunctionTransformer(rounding2))])  # Pipeline(steps=[('rounding', FunctionTransformer(rounding))])#
preprocessor_name = "preprocessor.p"
post_processer= StandardScaler(with_mean=True, with_std=True)#MinMaxScaler(feature_range=(-1, 1))


In [ ]:
# def smoothing_data(input: pd.DataFrame, window= 240):
#     shift_value= int(-window/2)
#     data= input.copy()
#     # data['t_bett_smoothed']= data['t_bett'].rolling(window= window).mean().shift(shift_value)
#     # data['t_motor_smoothed']= data['t_motor'].rolling(window= window).mean().shift(shift_value)
#     # data['t_spindle_smoothed']= data['t_spindle'].rolling(window= window).mean().shift(shift_value)
#     # data[str(output_variable+'_smoothed')]= data[output_variable].rolling(window= window).mean().shift(shift_value)
#     # data['t_bett']= data['t_bett'].rolling(window= window).mean().shift(shift_value)
#     # data['t_motor']= data['t_motor'].rolling(window= window).mean().shift(shift_value)
#     # data['t_spindle']= data['t_spindle'].rolling(window= window).mean().shift(shift_value)
#     data[smoothed_output]= data[output_variable].rolling(window= window).mean().shift(shift_value)
#     data= data[ window+1:-window]###Remove nan
#     return data
#     #return input
def smoothing_data(input: pd.DataFrame, window= 240):
    return input

In [ ]:
def filter_files():
    for file in glob.glob('*.csv'):
        if file == 'combined.csv':
            continue
        print('Current File: ', file)
        df = pd.read_csv(file)
        signals = df
        # print('Nans: ',signals.isnull().values.any())
        # print('places: ', signals.loc[pd.isnull(signals).any(1), :].index.values)
        prev_time=None
        drop_list= []
        ####Clean redundant data
        print('Before Filtering, ', len(signals))
        for index,row in signals.iterrows():
            current_time= datetime.strptime(row['date'], "%Y-%m-%d %H:%M:%S.%f")
            if prev_time is None:
                prev_time= current_time
            elif (current_time - prev_time).total_seconds() < 5:
                drop_list.append(datetime.strftime(current_time, "%Y-%m-%d %H:%M:%S.%f"))
                df= df[df['date'] != row['date']]
                current_time= prev_time
            prev_time= current_time
        #################Smoothing########################
        df.to_csv(path_or_buf=str('filtered_'+file))
        signals= df
        print('After Filtering, ', len(signals), ', deleted= ', len(drop_list))

Data Analysis Block

In [ ]:
data_mode= 'all_smooth'

In [ ]:
#source_dir= 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\Edgebox\\Versuchsdaten\\5s_data\\'
sn.set(font_scale=1.8)
working_dir = source_dir
os.chdir(working_dir+'validate_Data\\')
training_data = None
targets = None
needed_columns = selected_Columns.copy()
needed_columns.append(output_variable)
min_len= 100000000000
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    df = pd.read_csv(file)
    print('Current File: ', file)
    #print('current len: ', len(df))
    #if min_len > len(df):
    #    min_len= len(df)
    #print('Min:', min_len)
    smoothed_signals= smoothing_data(df, window= 24)
    signals= smoothed_signals[needed_columns]
    #signals.t_motor= signals.t_motor.round()
    if training_data is None:
        training_data = signals
    else:
        training_data = pd.concat([training_data, signals],axis=0, ignore_index= True)
    training_data= training_data[needed_columns]
    print(len(training_data))
training_data.reset_index(inplace=True)
training_data= training_data[needed_columns]
print(training_data.columns)
print('Nans: ',training_data.isnull().values.any())
print('places: ', training_data.loc[pd.isnull(training_data).any(1), :].index.values)
print(len(training_data))

# correlations = training_data.corr()
# fig, ax = plt.subplots(figsize=(15, 15))
# plt.title('Heat-Map für die Korrelationsmatrix')
# sn.heatmap(correlations, annot=True, vmin=-1, vmax=1, cmap='magma', annot_kws={"size": 26, 'color': 'black'})
# plt.show()
# fig, ax= plt.subplots(figsize=(15,15))
# plt.plot(training_data['t_spindel'],label= 't_spindel')
# plt.plot(training_data['t_motor'],label= 't_motor')
# plt.plot(training_data['t_spindel_vor'],label= 't_spindel_vor')
# plt.plot(training_data['t_spindel_rueck'],label= 't_spindel_rueck')
# plt.plot(training_data['t_raum'],label= 't_raum')
# plt.plot(training_data['M8']*50,label= 'M8')
# plt.plot(training_data['M7']*50,label= 'M7')
# plt.plot(training_data['M127']*50,label= 'M127')
# plt.plot(training_data['M121']*50,label= 'M121')
# plt.plot(training_data['t_kss_tank'],label= 't_kss_tank')
# plt.legend()
# plt.grid()
# plt.show()
#selected_columns= needed_columns
# y_axis_names = needed_columns
# fig = make_subplots(rows=len(needed_columns)+1, cols=1, shared_xaxes=True,
#                     print_grid=True, subplot_titles=needed_columns, vertical_spacing=0.02)
# for i in range(len(needed_columns)):
#     # x= training_data['start'],
#     fig.add_trace(go.Line(y=training_data[needed_columns[i]]), row=i+1, col=1)
#     fig.update_yaxes(title_text=y_axis_names[i], row=i+1, col=1)
# fig.add_trace(go.Line(y=targets), row=len(needed_columns) +
#               1, col=1)  # x= training_data['start'],
# fig.update_yaxes(title_text=output_variable, row=len(needed_columns), col=1)
# fig.update_layout(height=1200, width=1200, title_text="LF-Data")
# fig.show()
# print('input shape: ', training_data.shape)
# print('output shape: ', targets.shape)


In [ ]:
images_analysis_path= str(analysis_path+'{}_{}')

In [ ]:
training_data.loc[pd.isnull(training_data).any(1), :].index.values

In [ ]:
ax= training_data.hist(figsize= (15,15))
fig= ax[0][0].get_figure()
plt.xlabel('values')
plt.ylabel('counts')
#plt.savefig(images_analysis_path.format(data_mode,'row_data_hist.png') )
plt.show()

In [ ]:
correlations= training_data.corr()
fig, ax = plt.subplots(figsize=(15, 15))
plt.title('Mode: '+ data_mode+' Heat-Map für die Korrelationsmatrix')
ax= sn.heatmap(correlations, annot=True, vmin=-1, vmax=1, cmap='magma', annot_kws={"size": 26, 'color': 'black'})
#plt.savefig(images_analysis_path.format(data_mode,'row_data_corr.png') )
plt.show()

In [ ]:
training_data.head(10)

In [ ]:
#Features Standardization
scaler= StandardScaler(with_mean= True, with_std= True)
scaled_data= scaler.fit_transform(training_data)

In [ ]:
scaled_data_df= pd.DataFrame(scaled_data, columns= list(training_data.columns))
scaled_data_df.head(10)

In [ ]:
ax= scaled_data_df.hist(figsize= (15,15))
fig= ax[0][0].get_figure()
#plt.savefig(images_analysis_path.format(data_mode,'scaled_data_hist.png') )
plt.show()

In [ ]:
np.where(np.asanyarray(np.isnan(scaled_data_df)))

In [ ]:
correlations= scaled_data_df.corr()
fig, ax = plt.subplots(figsize=(15, 15))
plt.title('Mode: '+ data_mode+' Heat-Map für die Korrelationsmatrix')
ax= sn.heatmap(correlations, annot=True, vmin=-1, vmax=1, cmap='magma', annot_kws={"size": 26, 'color': 'black'})
#plt.savefig(images_analysis_path.format(data_mode,'scaled_data_corr.png') )
plt.show()

In [ ]:
# pca=PCA(n_components=7)
# data_decomposed= pca.fit_transform(scaled_data_df[['t_bett',	't_spindle','t_motor',	'M8',	'M121',	'M127','M7']])
# print('explanation_ratio= ',  pca.explained_variance_ratio_, 'precision: ', pca.get_precision())
# data_decomposed_df= pd.DataFrame(data_decomposed)
# #data_decomposed_df['org_label']= training_data[output_variable]
# #data_decomposed_df['scaled_label']= scaled_data_df[output_variable]
# data_decomposed_df['scaled_smoothed_label']= scaled_data_df[output_variable]
# ax= data_decomposed_df.hist(figsize= (15,15))
# fig= ax[0][0].get_figure()
# plt.savefig(images_analysis_path.format(data_mode,'decomp_data_hist.png') )
# plt.show()

In [ ]:
# correlations= data_decomposed_df.corr()
# fig, ax = plt.subplots(figsize=(25, 25))
# plt.title('Mode: '+ data_mode+' Heat-Map für die Korrelationsmatrix')
# ax= sn.heatmap(correlations, annot=True, vmin=-1, vmax=1, cmap='magma', annot_kws={"size": 26, 'color': 'black'})
# plt.savefig(images_analysis_path.format(data_mode,'decomp_data_corr.png') )
# plt.show()

Loading Training Data

In [ ]:
# Extract Data source from all signals and create Training dataset
working_dir = source_dir
os.chdir(working_dir)
results: np.ndarray = None
targets: np.ndarray = None
pipeline = None
reset = True
training_data = None
output= None
vis_columns= []
batch_size= 1000000000000000000
#selected_Columns= ['t_motor','t_spindel', 'drhz']
for file in glob.glob('*.csv'):
    print('Current File: ', file)
    df = pd.read_csv(file)
    
    smoothed_signals= smoothing_data(df, window= 24)
    # fig = make_subplots(rows=1, cols=1, shared_xaxes=True,
    #                 print_grid=True,  vertical_spacing=0.02)
    # fig.add_trace(go.Line(y=smoothed_signals[output_variable], name='smoothed_signal Loss'), row=1, col=1)
    # fig.add_trace(go.Line(y=smoothed_signals[output_variable].ewm(alpha=0.1).mean(), name='Origial_signal'), row=1, col=1)
    # fig.update_xaxes(title_text='Epochs', row=1, col=1)
    # fig.update_yaxes(title_text='Max_Loss', row=1, col=1)
    # fig.update_layout(height=900, width=900, title_text='training_curve')
    # fig.show()
    print('signals size= ', len(smoothed_signals))
    if smoothed_signals.isnull().values.any():
        print ('with nan')
    if batch_size > len(smoothed_signals):
        batch_size= len(smoothed_signals)
    if training_data is None:
        output= smoothed_signals[output_variable]
        training_data = smoothed_signals[selected_Columns]
    else:
        output= pd.concat([output, smoothed_signals[output_variable]], axis=0, ignore_index= True)
        training_data = pd.concat([training_data, smoothed_signals[selected_Columns]], axis=0, ignore_index= True)
    training_data= training_data[selected_Columns]
    print(len(training_data),' ',len(output))
# vis_columns= selected_Columns.copy()
# #vis_columns.append(str(output_variable+'_smoothed'))
# #training_data[output_variable]= output.tolist()
# testdf= training_data[vis_columns]
# correlations= pd.DataFrame(testdf, columns= vis_columns).corr()
# fig, ax = plt.subplots(figsize=(15, 15))
# plt.title('Mode: '+ data_mode+' Heat-Map für die Korrelationsmatrix')
# ax= sn.heatmap(correlations, annot=True, vmin=-1, vmax=1, cmap='magma', annot_kws={"size": 26, 'color': 'black'})
# plt.show()
print('Length of training data: ', len(training_data))
training_data= training_data.to_numpy()
print('Input Nans: ',np.isnan(training_data).any())
print('Possible batch_size= ', batch_size)
###################################################################################################################################


Generate Preprocessing Pipeline

In [ ]:
pipeline= None
results: np.ndarray = None
targets: np.ndarray = None
if pipeline is None:
    try:
        if not reset:
            pipeline = load(str(preprocessor_dir + preprocessor_name))
            print('preprocessor is loaded successfully')
        else:
            pipeline = proposed_pipline
            pipeline = pipeline.fit(training_data)
            dump(pipeline, str(preprocessor_dir + preprocessor_name))
            print('preprocessor is saved successfully')
            print('explained_variance_ratio_= ', pipeline.steps[3][1].explained_variance_ratio_)
    except FileNotFoundError as err:
        pipeline = proposed_pipline
        pipeline = pipeline.fit(training_data)
        dump(pipeline, str(preprocessor_dir + preprocessor_name))
        initial_type = [('float_input', FloatTensorType(
            [None,  training_data.shape[1]]))]
        onx = convert_sklearn(pipeline, initial_types=initial_type)
        with open(str(preprocessor_dir + "preprocessor.onnx"), "wb") as f:
            f.write(onx.SerializeToString())
        print('preprocessor is saved successfully')

Load and transform training data

In [ ]:
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    print('Current File: ', file)
    df = pd.read_csv(file)
    smoothed_signals= smoothing_data(df, window= 24)
    print('signals size= ', len(smoothed_signals))
    if smoothed_signals.isnull().values.any():
        print ('with nan')
    labels = (1000* smoothed_signals[output_variable]).round(3)#(1000* df[output_variable]).round(3)#.round(3)#(1000 * df[str('smoothed_' + output_variable)]).round(3)#1000* df[output_variable].round(3)#)
    smoothed_signals = smoothed_signals[selected_Columns]
    smoothed_signals = smoothed_signals.to_numpy()
    print('shape before transformation: ', smoothed_signals.shape)
    signals_tranformed = pipeline.transform(smoothed_signals)
    partitions, target = generateDataSource(signal_input=signals_tranformed, input_columns=selected_Columns, output_length=1, signal_output=labels, window=window, shift=shift, sample_rate=sampling_rate)
    if results is None:
        results = partitions
        targets = target
    else:
        results = np.concatenate((results, partitions), axis=0)
        targets = np.concatenate((targets, target), axis=0)
    print('input: ', results.shape, ' output:', targets.shape)
print(targets.min(), '    ',targets.max())
targets_post= post_processer.fit_transform(targets.reshape(-1,1))
dump(post_processer,str(preprocessor_dir+'postprocesser.p') )
print('input: ', results.shape, ' output:', targets_post.shape)
print(targets_post.min(), '    ',targets_post.max())
print('Input Nans: ',np.isnan(results).any())
print('Output Nans: ',np.isnan(targets_post).any())

Load Testing Data

In [ ]:
# Extract Data source from all signals
working_dir = testdata_dir
os.chdir(working_dir)
test_results: np.ndarray = None
test_targets: np.ndarray = None
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    print('Current File: ', file)
    df = pd.read_csv(file)
    signals= smoothing_data(df, window= 24)
    print('signals size= ', len(signals))
    if signals.isnull().values.any():
        print ('with nan')
    labels = (1000* signals[output_variable]).round(3)#(1000* df[output_variable]).round(3)#.round(3)#(1000 * df[str('smoothed_' + output_variable)]).round(3)#1000* df[output_variable].round(3)#)
    signals = signals[selected_Columns]
    signals = signals.to_numpy()
    print('shape before transformation: ', signals.shape)
    signals_transformed = pipeline.transform(signals)
    partitions, target = generateDataSource(signal_input=signals_transformed, input_columns=selected_Columns, output_length=1, signal_output=labels, window=window, shift=shift, sample_rate=sampling_rate)
    if test_results is None:
        test_results = partitions
        test_targets = target
    else:
        test_results = np.concatenate((test_results, partitions), axis=0)
        test_targets = np.concatenate((test_targets, target), axis=0)
    print('input: ', test_results.shape, ' output:', test_targets.shape)
print(test_targets.min(), '    ',test_targets.max())
test_targets_post= post_processer.transform(test_targets.reshape((-1,1)))
print('input: ', test_results.shape, ' output:', test_targets_post.shape)
print(test_targets_post.min(), '    ',test_targets_post.max())
print('Input Nans: ',np.isnan(test_results).any())
print('Output Nans: ',np.isnan(test_targets_post).any())

In [ ]:
def map(old_value, old_min, old_max, new_max, new_min):
    new_value= ( (old_value - old_min) / (old_max - old_min) ) * (new_max - new_min) + new_min
    return new_value

In [ ]:
def calculate_weights(y_train: np.ndarray, occurance_threshold= 0):
    old_min = 0
    old_max = 0
    y_train_rounded = np.round(y_train,decimals=3)
    N = len(y_train_rounded)
    y_train_rounded = np.reshape(y_train_rounded, newshape=(N,))
    weights = np.ones(shape=(N,),dtype= np.float32)
    for i in range(0, N):
        current_value = y_train_rounded[i]
        occurences = np.count_nonzero(y_train_rounded == current_value)
        if occurences <= occurance_threshold:
             weights[i]=  N/(occurences + 1 )#N/(occurences + y_train_rounded[i])#N/occurences # 0
        else:
            weights[i] =  N/(occurences + 1)#(8 if y_train_rounded[i] < 20 else 1)
    old_min= weights.min()
    old_max= weights.max()
    new_min = 1
    new_max = 1.1
    #weights_scaled= np.apply_along_axis(map, 1, weights)
    weights_scaled= np.asanyarray([ map(weights[i], old_min, old_max, new_max, new_min) for i in range(0, weights.shape[0])]).reshape(weights.shape)
    #weights = weights - weights.min() + 1
    #weights = (weights - weights.min())/(weights.max() - weights.min())
    fig= make_subplots(rows=2,cols=1,shared_xaxes= True, print_grid= True,  vertical_spacing=0.02)
    fig.add_trace(go.Line(y=weights_scaled,name='weight of Labels')#visualisation_selected_Columns[-2])
    , row= 1, col= 1)
    fig.add_trace(go.Line(y=np.reshape(y_train, newshape=(N,)),name='Labels')#visualisation_selected_Columns[-2])
    , row= 2, col= 1)
    # fig.update_yaxes(title_text= 'weight', row= 1, col= 1)
    # fig.update_xaxes(title_text= 'label value', row= 1, col= 1)
    fig.update_layout(height=900, width=900, title_text= 'weight of Labels')
    fig.show()

    return weights_scaled


In [ ]:
def build_Model_LSTM(input_shape, activation_Function = selu,
    dropout = 0.0,
    l1_v = 0.00,
    l2_v = 0.00,
    structure= [],
    n_hidden_layers= 1,
    n_units=1,
    optimizer= 'Nadam'):
    #structure=  [n_units for i in range(0,n_hidden_layers)]#[50,50,40,40,30,30,20]##   
    unroll = False
    kernal_init = 'he_normal'#RandomUniform()  # 
    model = Sequential()
    model.add(Input(shape=input_shape))
    for i in range(1,len(structure)+1):
        layer_size= structure[i-1]
        if i == len(structure):
            model.add(LSTM(layer_size,stateful= False,return_sequences=False,unroll=unroll,kernel_initializer= kernal_init, dropout=dropout, kernel_regularizer=L1L2(l1=l1_v, l2=l2_v)))
        else:
            model.add(LSTM(layer_size,stateful= False, return_sequences=True,unroll=unroll, kernel_initializer= kernal_init,dropout=dropout,  kernel_regularizer=L1L2(l1=l1_v, l2=l2_v)))
        #model.add(BatchNormalization())
        model.add(Activation(activation=activation_Function))
        #model.add(BatchNormalization())
        #model.add(PReLU())
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer, loss= special_loss, metrics=[ max_loss])#'Adagrad'
    return model

In [ ]:
x_training,x_testing,y_training,y_testing= train_test_split(results, targets, test_size=0.1,random_state= 42, shuffle=False)

In [ ]:
# Training Block
checkpoint = ModelCheckpoint(filepath=str(model_dir+'best_model.h5'), monitor='val_loss', verbose=1,
                             save_best_only=True, save_weights_only=False, mode='min', save_freq='epoch')
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
earlyStopping = EarlyStopping(
    monitor='val_loss', mode='min', patience=500, verbose=1)

In [ ]:
params= {'activation_Function': ['relu', 'tanh', 'elu', 'selu'],
    'n_units':range(5,50,5),
    'n_hidden_layers': range(1,6,1),
    'optimizer': ['Nadam', 'Adamax', 'Adagrad'],
    'epochs': range(100,600,100),
    'batch_size': range(500,2000,100)}

In [ ]:
"""
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import make_scorer
scoring_function= make_scorer(score_func= max_loss, greater_is_better= False, needs_proba= False,)
#'dropout': np.linspace(start=0, stop=0.5, num=10),'l1_v': np.linspace(start=0, stop=0.001, num=10),'l2_v': np.linspace(start=0, stop=0.001, num=10),
params= {
    'n_units':range(10,25,5),
    'n_hidden_layers': range(1,4,1)
   }
model_wrapper= KerasRegressor(build_fn= build_Model_LSTM, input_shape= (results.shape[1], results.shape[2]),activation_Function= 'relu', n_units= 1, n_hidden_layers= 1, optimizer= 'Nadam',  epochs= 500, batch_size= 50, callbacks=[earlyStopping], verbose=2, workers=32, use_multiprocessing=True)
random_grid= RandomizedSearchCV(estimator= model_wrapper, param_distributions= params, n_iter=100, scoring= mean_absolute_error, n_jobs= -1, cv=10,verbose=3)
random_grid= GridSearchCV(estimator= model_wrapper,param_grid= params, scoring= scoring_function, n_jobs= -1, cv=10,verbose=3)
fitted= random_grid.fit(X= results, y= targets, shuffle= False,batch_size= 1200)
print(random_grid.best_score_)
"""


In [ ]:

weights = calculate_weights(y_train=y_training, occurance_threshold= 5)
model = build_Model_LSTM( (x_training.shape[1], x_training.shape[2]),activation_Function = selu,
    dropout = 0.0,
    l1_v = 0.0000,
    l2_v = 0.0000,
    n_hidden_layers= 2,
    n_units=10,
    structure=[5],
    optimizer= 'Nadam')
reset= True##############################################################################################################
try:
    if not reset:
        #model.set_weights(model_w)
        #model.load_weights(filepath=weights_path)
        print('Previous weights loaded Successfully')
except:
    print('No Previous weights')
print('input shape ', model.input_shape)
print(model.output_shape)
print(model.summary())
summary = model.fit(x=x_training, y=y_training, shuffle=False, batch_size= int(batch_size/2),  epochs=15000, validation_data=(x_testing, y_testing),  #int(batch_size * 0.5)
                    callbacks=[earlyStopping], verbose=2, workers=32, use_multiprocessing=True, sample_weight=weights)


In [ ]:
model.save(str(model_dir+'model.h5'), save_format='h5')
input_signature = [tf.TensorSpec([None, results.shape[1], results.shape[2]], tf.float32, name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(
    model=model, input_signature=input_signature, opset=10)
onnx.save_model(onnx_model, str(model_dir+'model.onnx'))
print(".onnx model saved successfully")

In [ ]:
model_w= model.get_weights()
#model.save_weights(weights_path, save_format='h5')l
#model.save_weights(weights_path, save_format='tf')
#model.save(str(model_dir+'testmodel.h5'), save_format='h5', overwrite=True)

In [ ]:
#model.save_weights(filepath=weights_path)
metric= 'mae_loss_loss'
metric_loss = summary.history['loss']
metric_val_loss = summary.history['val_loss']
max_loss_res= summary.history['max_loss']
val_max_loss_res= summary.history['val_max_loss']
#mae_loss= summary.history['mean_absolute_error']
#val_mae_loss= summary.history['val_mean_absolute_error']
fig = make_subplots(rows=1, cols=1, shared_xaxes=True,
                    print_grid=True,  vertical_spacing=0.02)
fig.add_trace(go.Line(y=max_loss_res, name='Training Max_Loss'), row=1, col=1)
fig.add_trace(go.Line(y=val_max_loss_res , name='Validation Max_Loss'), row=1, col=1)
fig.add_trace(go.Line(y=metric_loss, name='Training {}'.format(metric)), row=1, col=1)
fig.add_trace(go.Line(y=metric_val_loss, name='Validation {}'.format(metric)), row=1, col=1)
#fig.add_trace(go.Line(y=mae_loss, name='Training {}'.format('mae')), row=1, col=1)
#fig.add_trace(go.Line(y=val_mae_loss, name='Validation {}'.format('mae')), row=1, col=1)
fig.update_xaxes(title_text='Epochs', row=1, col=1)
fig.update_yaxes(title_text='Loss', row=1, col=1)
fig.update_layout(height=900, width=900, title_text='Training Curve: mae vs. {}'.format(metric))
fig.show()
pio.write_image(fig, str(image_path+'mae_vs_{}.png'.format(metric)), format='png')

In [ ]:
# #####Save Model as onnx from weights
# from keras.models import load_model
# import tf2onnx
# import onnx
# import tensorflow as tf
# model= Sequential()
# model= load_model(filepath= str(model_dir+'model.h5'), compile= False)
# model.load_weights(weights_path)
# model.save(str(model_dir+'best_model.h5'), save_format='h5')
# input_signature= [tf.TensorSpec([None,results.shape[1], results.shape[2]],tf.float32, name= 'x')]
# onnx_model, _= tf2onnx.convert.from_keras(model= model, input_signature= input_signature, opset= 10)
# onnx.save_model(onnx_model, str(model_dir+'Best_model.onnx'))
# print (".onnx model saved successfully")


In [ ]:
import numpy as np
tolerance = 5
x_steps = np.linspace(0, 50, 10)
lowerline = x_steps - tolerance * np.sin(np.pi/4)
upperline = x_steps + tolerance * np.sin(np.pi/4)
# Create Tolerance Lines
tolerance = 10
x_steps = np.linspace(0, 50, 10)
lowerline_actual = x_steps - tolerance * np.sin(np.pi/4)
upperline_actual = x_steps + tolerance * np.sin(np.pi/4)


In [ ]:
import numpy as np
tolerance = 5
x_steps = np.linspace(0, 50, 10)
lowerline = x_steps - tolerance * np.sin(np.pi/4)
upperline = x_steps + tolerance * np.sin(np.pi/4)
# Create Tolerance Lines
tolerance = 10
x_steps = np.linspace(0, 50, 10)
lowerline_actual = x_steps - tolerance * np.sin(np.pi/4)
upperline_actual = x_steps + tolerance * np.sin(np.pi/4)

In [ ]:
import numpy as np
from keras.models import load_model
#from keras.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
# model= build_Model(X_train= results,y_train= targets)
# model.load_weights(filepath= weights_path)
# model.save(str(model_dir+'model.h5'), save_format='h5')
#model = Sequential()
#model = load_model(filepath=str(model_dir+'model.h5'), compile=False)
#pipeline = load(str(preprocessor_dir+preprocessor_name))
print(model.summary())
print(model.input_shape)
# pred_train= model.predict(x_train)
# print ('mse= ',np.mean(mean_squared_error(y_train,pred_train)), ' mae= ', np.mean(mean_absolute_error(y_train,pred_train)), ' mape= ', np.mean(mean_absolute_percentage_error(y_train,pred_train)))
# pred_test= model.predict(x_test)
# print ('mse= ',np.mean(mean_squared_error(y_test,pred_test)), ' mae= ', np.mean(mean_absolute_error(y_test,pred_test)), ' mape= ', np.mean(mean_absolute_percentage_error(y_test,pred_test)))
pred = model.predict(results)
#print (' For All Data mse= ',mean_squared_error(targets,pred), ' mae= ', mean_absolute_error(targets,pred), ' mape= ', mean_absolute_percentage_error(targets,pred))
#pred= post_processer.inverse_transform(pred)
#org_targets= post_processer.inverse_transform(targets_post)
fig, ax = plt.subplots(figsize=(15, 15))
plt.title('Results of LSTM Algorithm')
plt.xlabel('True')
plt.ylabel('Predicted')
pred.reshape((-1,))
plt.plot(x_steps, upperline, label='Upper Bound', color='green')
plt.scatter(targets, pred, label='predictions_train', color='blue')
plt.plot(x_steps, lowerline, label='lower Bound', color='green')
plt.plot(x_steps, x_steps, label='Optimal line')
plt.plot(x_steps, lowerline_actual, label='Actual lower Bound', color='red')
plt.plot(x_steps, upperline_actual, label='Actual Upper Bound', color='red')
plt.legend()
plt.savefig(str(image_path+'training_results.png'))
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
plt.title('Results of LSTM Algorithm')
plt.xlabel('True')
plt.ylabel('Predicted')
pred.reshape((-1,))
plt.plot(x_steps, upperline, label='Upper Bound', color='green')
plt.scatter(targets, pred, label='predictions_train', color='blue')
plt.plot(x_steps, lowerline, label='lower Bound', color='green')
plt.plot(x_steps, x_steps, label='Optimal line')
plt.plot(x_steps, lowerline_actual, label='Actual lower Bound', color='red')
plt.plot(x_steps, upperline_actual, label='Actual Upper Bound', color='red')
plt.legend()
plt.savefig(str(image_path+'training_results.png'))
plt.show()

In [ ]:
# #####Save Model as onnx from weights
# from keras.models import load_model
# import tf2onnx
# import onnx
# import tensorflow as tf
# model= Sequential()
# model= load_model(filepath= str(model_dir+'model.h5'), compile= False)
# model.load_weights(weights_path)
# model.save(str(model_dir+'best_model.h5'), save_format='h5')
# input_signature= [tf.TensorSpec([None,results.shape[1], results.shape[2]],tf.float32, name= 'x')]
# onnx_model, _= tf2onnx.convert.from_keras(model= model, input_signature= input_signature, opset= 10)
# onnx.save_model(onnx_model, str(model_dir+'Best_model.onnx'))
# print (".onnx model saved successfully")


In [ ]:
import numpy as np
from keras.models import load_model
#from keras.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
#model = Sequential()
#model = load_model(filepath=str(model_dir+'model.h5'), compile=False)
#pipeline = load(str(preprocessor_dir+preprocessor_name))
# print(model.summary())
# pred_train= model.predict(x_train)
# print ('mse= ',np.mean(mean_squared_error(y_train,pred_train)), ' mae= ', np.mean(mean_absolute_error(y_train,pred_train)), ' mape= ', np.mean(mean_absolute_percentage_error(y_train,pred_train)))
# pred_test= model.predict(x_test)
# print ('mse= ',np.mean(mean_squared_error(y_test,pred_test)), ' mae= ', np.mean(mean_absolute_error(y_test,pred_test)), ' mape= ', np.mean(mean_absolute_percentage_error(y_test,pred_test)))
pred = model.predict(test_results)
#print (' For All Data mse= ',mean_squared_error(targets,pred), ' mae= ', mean_absolute_error(targets,pred), ' mape= ', mean_absolute_percentage_error(targets,pred))
fig, ax = plt.subplots(figsize=(15, 15))
plt.title('Ergebnisse des KI-Modells')
plt.xlabel('Gemessene Verlagerung (Welle) Mm')
plt.ylabel('Vorhersage der Verlagerung (Welle) Mm')
plt.plot(x_steps, upperline, label='5 Mm Obere Grenze', color='green')
plt.scatter(test_targets, pred, label='Vorhersage', color='blue')
# plt.scatter(y_train,pred_train, label= 'predictions_train', color= 'blue')
# plt.scatter(y_test,pred_test, label= 'predictions_test', color= 'magenta')
plt.plot(x_steps, lowerline, label='5 Mm Untere Grenze', color='green')
plt.plot(x_steps, x_steps, label='Optimale Linie')
plt.plot(x_steps, lowerline_actual, label='10 Mm Obere Grenze', color='red')
plt.plot(x_steps, upperline_actual, label='10 Mm Untere Grenze', color='red')
plt.legend()
plt.savefig(str(image_path+'testing_results.png'))
plt.show()


In [ ]:

# Comparasion between Komp and Z_tcp_ok
current_dir = 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/row_files/combines/smoothed/combined_file/'
file = str(current_dir + 'combined.csv')
df = pd.read_csv(file)
z_tcp_ok = df[output_variable]
komp = -0.1*df['komp']
print('mse= ', np.mean(mean_squared_error(z_tcp_ok, komp)),
      ' mae= ', np.mean(mean_absolute_error(z_tcp_ok, komp)))
fig, ax = plt.subplots(figsize=(15, 15))
plt.title('Komp and Z_TCP_OK')
plt.xlabel('True')
plt.ylabel('Komp')
plt.grid()
plt.plot(x_steps, upperline, label='Upper Bound', color='green')
plt.scatter(z_tcp_ok, komp, label='komp', color='blue')
plt.plot(x_steps, lowerline, label='lower Bound', color='green')
plt.plot(x_steps, x_steps, label='Optimal line')
plt.plot(x_steps, lowerline_actual, label='Actual lower Bound', color='red')
plt.plot(x_steps, upperline_actual, label='Actual Upper Bound', color='red')
plt.legend()
plt.show()


In [ ]:
#image_path= str(source_dir + 'results/')
#selected_Columns= ['t_spindel', 't_motor']
# Validate the model based on validataion data
data_path = str(source_dir + 'validate_Data/')
#image_path= str(source_dir + 'results/')
#pipeline = load(str(preprocessor_dir+preprocessor_name))
print('preprocessor is loaded successfully')
#model = Sequential()
#model = load_model(str(model_dir+'model.h5'), compile=False)
print('Model is loaded successfully')
os.chdir(data_path)
errors = []
error_df = pd.DataFrame()
scatter_mode= 'lines'
visualisation_selected_Columns= ['t_bett', 't_motor', 't_spindel', 'M8','M121', 'M127', 'M7',output_variable, 'prediction_Error' ]
for file in glob.glob('*.csv'):
    df = pd.read_csv(file)
    #df = remove_peeks(df, output_variable, 4)
    print('current File: ', file)
    #df['t_spindel']= df['t_spindle']
    #signals = preprocess_data(df, kuelung=['M8', 'M121', 'M127', 'M7'])
    
    signals = df[selected_Columns]
    
    signals = signals.to_numpy()
    signals = pipeline.transform(signals)
    # (1000 * df[str('smoothed_' + output_variable)]).round(2)#
    output = (1000* df[output_variable]).round(3)#(1000 * df[str('smoothed_' + output_variable)]).round(3)
    partitions, target = generateDataSource(
        signal_input=signals, input_columns=selected_Columns, output_length=1, signal_output=output, window=window, shift=shift, sample_rate=sampling_rate)
    target = target.reshape((-1,))
    predictions = model.predict(partitions)
    predictions = np.reshape(predictions, newshape=target.shape)
    diff = np.abs(target - predictions)
    mae_v = ' mae= ' + str(np.mean(mean_absolute_error(target, predictions).round(2))) +' Micro_Meter '  
    max_v = ' max= ' + str(np.max(np.abs(target - predictions).round(2))) +' Micro_Meter '  

    print(mae_v)
    print(max_v)
    fig, ax = plt.subplots(figsize=(15, 15))
    plt.xlabel('Zeit')
    plt.ylabel('Verlagerung (Welle) Mm')
    plt.title(str(file + mae_v + max_v))
    plt.plot(target, label='gemessene Verlagerung (Welle)')
    plt.plot(predictions, label= 'prediction')
    plt.grid()
    plt.legend()
    plt.show()
    

In [ ]:
######################
# Visualization of lf-data with predicted results
visualisation_selected_Columns = selected_Columns.copy()
visualisation_selected_Columns.append(output_variable)
visualisation_df = df[window-1:]
visualisation_df['predictions'] = predictions
fig = make_subplots(rows= 1, cols=1, shared_xaxes=True, print_grid=True,  vertical_spacing=0.02)
###verlagerung
fig.add_trace(go.Line(y= 1000 * visualisation_df[output_variable], name='Echte z_welle'), row= 1, col=1)
fig.add_trace(go.Line(y= visualisation_df['predictions'], name='KI_z_welle'), row= 1, col=1)
fig.update_yaxes(title_text='Verlagerung Mic-Meter', row= 1, col=1)
fig.update_xaxes(title_text='Zeit (1= 2 Minuten)', row=1, col=1)
fig.update_layout(height=900, width=1200, title_text=str(file + mae_v + max_v))
fig.show()
    #pio.write_image(fig, str(image_path + file+'_Verlagerung.png'), format='png')


Apply predictions on the experiments of validation and save results as images

In [ ]:
#image_path= str(source_dir + 'results/')
#selected_Columns= ['t_spindel', 't_motor']
# Validate the model based on validataion data
data_path = str(source_dir + 'validate_Data/')
#image_path= str(source_dir + 'results/')
#pipeline = load(str(preprocessor_dir+preprocessor_name))
print('preprocessor is loaded successfully')
#model = Sequential()
#model = load_model(str(model_dir+'model.h5'), compile=False)
#post_processer= load(str(preprocessor_dir+'postprocesser.p'))
print('Model is loaded successfully')
os.chdir(data_path)
errors = []
error_df = pd.DataFrame()
scatter_mode= 'lines'
visualisation_selected_Columns= ['t_bett', 't_motor', 't_spindle', 'M8','M121', 'M127', 'M7',output_variable, 'prediction_Error' ]
for file in glob.glob('*.csv'):
    df = pd.read_csv(file)
    #df = remove_peeks(df, output_variable, 4)
    print('current File: ', file)
    df= smoothing_data(df, window= 24)
    signals = df[selected_Columns]
    signals = signals.to_numpy()
    signals = pipeline.transform(signals)
    # (1000 * df[str('smoothed_' + output_variable)]).round(2)#
    output = (1000* df[output_variable]).round(3)#(1000 * df[str('smoothed_' + output_variable)]).round(3)
    partitions, target = generateDataSource(
        signal_input=signals, input_columns=selected_Columns, output_length=1, signal_output=output, window=window, shift=shift, sample_rate=sampling_rate)
    target = target.reshape((-1,1))
    predictions = model.predict(partitions)
    predictions = np.reshape(predictions, newshape=target.shape)
    #predictions= post_processer.inverse_transform(predictions)
    #target= post_processer.inverse_transform(target)
    diff = np.abs(target - predictions)
    mae_v = ' mae= ' + str(np.mean(mean_absolute_error(target, predictions))) +' Micro_Meter '  
    max_v = ' max= ' + str(np.max(np.abs(target - predictions))) +' Micro_Meter '  
    print(mae_v)
    print(max_v)
    ######################
    # Visualization of lf-data with predicted results
    visualisation_selected_Columns = selected_Columns.copy()
    visualisation_selected_Columns.append(output_variable)
    print(abs(len(df)-len(partitions))
    visualisation_df = df[abs(len(df)-len(partitions)):]
    visualisation_df[str('smoothed_' + output_variable)] = output
    visualisation_df['predictions'] = predictions
    fig = make_subplots(rows= 9, cols=1, shared_xaxes=True, print_grid=True,  vertical_spacing=0.02)
    # ##t_bett
    # fig.add_trace(go.Line(y=visualisation_df['t_bett'], name='t_bett'), row= 1, col=1)
    # fig.update_yaxes(title_text='t_bett C°', row= 1, col=1)
    # ##t_bett
    # fig.add_trace(go.Line(y=visualisation_df['drhz'], name='drhz'), row= 1, col=1)
    # fig.update_yaxes(title_text='drhz RPM', row= 1, col=1)
    ###t_motor
    fig.add_trace(go.Line(y=visualisation_df['t_motor'], name='t_motor'), row= 2, col=1)
    fig.update_yaxes(title_text='t_motor C°', row= 2, col=1)
    ###t_spindel
    fig.add_trace(go.Line(y=visualisation_df['t_spindle'], name='t_spindle'), row= 3, col=1)
    fig.update_yaxes(title_text='t_spindel C°', row= 3, col=1)
    ###M8
    fig.add_trace(go.Line(y=visualisation_df['M8'], name='M8'), row= 4, col=1)
    fig.update_yaxes(title_text='M8', row= 4, col=1)
    ###M121
    fig.add_trace(go.Line(y=visualisation_df['M121'], name='M121'), row= 5, col=1)
    fig.update_yaxes(title_text='M121', row= 5, col=1)
    ###M127
    fig.add_trace(go.Line(y=visualisation_df['M127'], name='M127'), row= 6, col=1)
    fig.update_yaxes(title_text='M127', row= 6, col=1)
    ###M7
    fig.add_trace(go.Line(y=visualisation_df['M7'], name='M7'), row= 7, col=1)
    fig.update_yaxes(title_text='M7', row= 7, col=1)
    ###verlagerung
    fig.add_trace(go.Line(y=  1000* visualisation_df[output_variable], name='z_welle_Soll'), row= 8, col=1)
    fig.add_trace(go.Line(y=  visualisation_df['predictions'], name='KI_z_welle_Ist'), row= 8, col=1)
   ##fig.add_trace(go.Line(y=  1000* df[output_variable], name='z_welle_Soll_smoothed'), row= 8, col=1)
    fig.update_yaxes(title_text='Verlagerung Mic-Meter', row= 8, col=1)
    ###Fehler
    experiment_errors = (1000* visualisation_df[output_variable] - visualisation_df['predictions']).to_list()
    fig.add_trace(go.Scatter( y=   experiment_errors, name='Prediction Error', mode= scatter_mode),  row= 9 , col= 1)
    ## Draw the tolerence +-5
    fig.add_trace(go.Scatter(y= np.full_like(experiment_errors,5), name='+5 Max Error', mode= scatter_mode),  row= 9 , col= 1)
    fig.add_trace(go.Scatter(y= np.full_like(experiment_errors,-5), name='-5 Min Error', mode= scatter_mode),  row= 9 , col= 1)
    fig.update_yaxes(title_text='Fehler Mic-Meter', row= 9, col=1)
    fig.update_xaxes(title_text='Zeit (1= 2 Minuten)', row=9, col=1)
    fig.update_layout(height=900, width=1200, title_text=str(file + mae_v + max_v))
    fig.show()
    pio.write_image(fig, str(image_path + file+'.png'), format='png')
    current_error_df = pd.DataFrame(
        {'verlagerung': visualisation_df['predictions'].to_list(), 'error': experiment_errors})
    if error_df is None:
        error_df = current_error_df
    else:
        error_df = pd.concat([error_df, current_error_df])
    errors.extend(experiment_errors)


In [ ]:
# Visualization of resulted Data set
import plotly.express as px
labels = error_df['verlagerung']
verlagerung_df = pd.DataFrame({'verlagerung': labels})
print(verlagerung_df.head(5))
fig = px.histogram(verlagerung_df, x='verlagerung', nbins=100)
mean = verlagerung_df['verlagerung'].mean()
std = verlagerung_df['verlagerung'].std()
fig.update_layout(height=900, width=900, title_text=str(
    'Mean= ' + str(mean) + ', std=' + str(std)))
fig.show()
pio.write_image(
    fig, str(image_path + 'distribution of displacement.png'), format='png')


In [ ]:
# Visualization of resulted Data set
import plotly.express as px
error_df['abs_error'] = error_df['error'].abs()
print(error_df.head(5))
fig = px.histogram(error_df, x='verlagerung',
                   y='abs_error', nbins=100, histfunc='avg')
mean = error_df['abs_error'].mean()
std = error_df['abs_error'].std()
fig.update_layout(height=900, width=900, title_text=str(
    'Mean= ' + str(mean) + ', std=' + str(std)))
fig.show()
pio.write_image(fig, str(
    image_path + 'distribution of average prediction error on the displacement.png'), format='png')


In [ ]:
import plotly.express as px
error_df = pd.DataFrame({'errors': errors})
print(error_df.head(5))
fig = px.histogram(error_df, x='errors', nbins=50)
mean = error_df['errors'].mean()
std = error_df['errors'].std()
fig.update_layout(height=900, width=900, title_text=str(
    'Mean= ' + str(mean) + ', std=' + str(std)))
fig.show()
pio.write_image(fig, str(
    image_path + 'Distribution of the prediction error.png'), format='png')


In [ ]:
# from skl2onnx import convert_sklearn
# from skl2onnx.common.data_types import FloatTensorType
# from sklearn.pipeline import Pipeline
# from joblib import load
# pipeline= load(str(preprocessor_dir+'preprocessor.p'))
# print(pipeline)
# initial_type = [('float_input', FloatTensorType([None, 4]))]
# onx = convert_sklearn(pipeline, initial_types=initial_type)
# with open(str(preprocessor_dir + "prepeocessor.onnx"), "wb") as f:
#     f.write(onx.SerializeToString())


In [ ]:
# from joblib import dump, load
# preprocessor= load(str(preprocessor_dir+"preprocessor.p"))
# dump(preprocessor,str(preprocessor_dir+"preprocessor.p"))
